1. Establish a connection between Python and the Sakila database.



In [3]:
import pandas as pd
import mysql.connector
from getpass import getpass
password=getpass()
cnx=mysql.connector.connect(user="root",password=password,host="localhost",database="sakila")
cnx.is_connected()

True

In [9]:
import pandas as pd


2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

*  engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
*  month: an integer representing the month for which rental data is to be retrieved.
*  year: an integer representing the year for which rental data is to be retrieved.
<br>
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [11]:
def rental_month(cursor,month,year):
    cursor=cnx.cursor()
    query=(f"""SELECT * FROM sakila.rental
                WHERE month(rental_date)={month} and year(rental_date)={year}""") 
    cursor.execute(query)
    results=cursor.fetchall() #results is a list of tuples, where each tuple is a row
    columns=[header[0] for header in cursor.description]
    rental_df=pd.DataFrame(results, columns=columns)
    return rental_df


In [14]:
rental_month_df=rental_month(cursor,month=5,year=2005)
rental_month_df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

In [20]:
def rental_count_month (df,month,year):
    df=df.groupby("customer_id").agg({"rental_id":"count"})
    df.columns=[f"rentals_{month}_{year}"]
    return df

In [21]:
rental_count_month_df=rental_count_month(df=rental_month_df,month=5,year=2005)
rental_count_month_df.head()

,rentals_5_2005
customer_id,
1,2
2,1
3,2
5,3
6,3


4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [33]:
rental_count_month_df1=rental_count_month(df=rental_month_df,month=5,year=2005)
rental_count_month_df2=rental_count_month(df=rental_month_df,month=8,year=2005)

def compare_rentals(df1,df2):
    df=pd.merge(left=df1, right=df2, left_on="customer_id", right_on="customer_id", how="outer")
    df["difference"]=df[df.columns[1]]-df[df.columns[0]]
    return df

In [35]:
compare_rentals(rental_count_month_df1,rental_count_month_df2)

,rentals_5_2005,rentals_8_2005,difference
customer_id,,,
1,2,2,0
2,1,1,0
3,2,2,0
5,3,3,0
6,3,3,0
...,...,...,...
594,4,4,0
595,1,1,0
596,6,6,0
